In [2]:
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas
import geopandas as gp
import maup
from gerrychain.metrics import mean_median, efficiency_gap, polsby_popper, compactness
import random

In [3]:
#Read in shapefile with geopandas
#The shapefile contains all significant information including the elections data, population dynamics  

df = gp.read_file('IL_tracts_projected32616_MOD_SingleParts_Data_Plan/IL_tracts_projected32616_MOD_SingleParts_Data_Plans3.shp') # + initial district assignments

# Create Graph object from gerrychain
graph = Graph.from_geodataframe(df)

df.head()

/Users/saeesbox/opt/anaconda3/lib/python3.8/site-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i


,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,...,POP20,VOTES_DEM,VOTES_REP,NHWHITEPOP,BLACKPOP,HISPLATPOP,PLAN_1%_It,PLAN_1%,PLAN1%500,geometry
0,17,019,010400,17019010400,104,Census Tract,G5020,S,465467504,536184,...,4498,462.825580,1506.036700,3919,103,253,12,2,13,"POLYGON ((395305.024 4472831.939, 395606.344 4..."
1,17,019,010601,17019010601,106.01,Census Tract,G5020,S,33296629,469058,...,5749,1010.775526,1787.485623,5141,41,201,12,15,1,"POLYGON ((375461.039 4453009.300, 375463.859 4..."
2,17,019,010500,17019010500,105,Census Tract,G5020,S,375283527,1269717,...,4593,543.500000,1640.166667,4287,23,79,12,2,1,"POLYGON ((375721.985 4453731.917, 375725.870 4..."
3,17,041,952200,17041952200,9522,Census Tract,G5020,S,9862639,25982,...,4668,577.277786,1404.979006,4251,26,158,15,15,1,"POLYGON ((389193.930 4404417.016, 389213.208 4..."
4,17,041,952300,17041952300,9523,Census Tract,G5020,S,273551600,370881,...,5395,286.505356,1095.829392,5140,6,145,15,15,1,"POLYGON ((373866.070 4405691.509, 373984.309 4..."


In [3]:
# Create Election object from gerrychain

election = Election('AVG', {'Dem': 'VOTES_DEM', 'Rep': 'VOTES_REP'})

'\nmyfile = open(\'ILvotedata.txt\', \'w\')\nmyfile.write("%s\n" % V)\nmyfile.write("%s\n" % votesA)\nmyfile.write("%s\n" % votesB)\nmyfile.close()\n'

In [4]:
# Create a GeographicPartition object from gerrychain (initial district plan)
# GeographicPartition automatically has updaters for perim, area, cut-edges, and more!

initial_partition = GeographicPartition(
    graph,
    assignment='PLAN_1%',
    updaters={
        'population': updaters.Tally('POP20', alias='population'),
        'nhwhitepop': updaters.Tally('NHWHITEPOP', alias='nhwhitepop'),
        'blackpop': updaters.Tally('BLACKPOP', alias='blackpop'),
        'latpop': updaters.Tally('HISPLATPOP', alias='latpop'),
        'AVG': election
    }
)

# print(initial_partition['AVG'].percents('Dem'))
# print(initial_partition['AVG'].wins('Dem'))
# print(len(initial_partition['cut_edges']))
# print(sum(initial_partition['perimeter'].values()))

In [5]:
# Set the ideal population
ideal_population = sum(initial_partition['population'].values())/len(initial_partition)

# Build proposal function
proposal = partial(recom,
                   pop_col='POP20',
                   pop_target=ideal_population,
                   epsilon=0.01,
                   node_repeats=2
                  )


In [6]:
# Bound the number of cut edges at 2 times the number of cut edges in the initial plan
compactness_bound = constraints.UpperBound(
    lambda p: len(p['cut_edges']),
    2*len(initial_partition['cut_edges'])
)

chain = MarkovChain(
    proposal=proposal,
    constraints=[
        # District populations must stay within 1% of equality
        constraints.within_percent_of_ideal_population(initial_partition, 0.01),
        compactness_bound
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=1000
    )

In [9]:

#Bound the number of cut edges to kstar times the original number. Done iteratively to the maximum
# possible extent

kset = [0.9, 0.8, 0.7, 0.6, 0.58, 0.56, 0.54, 0.52, 0.51, 0.5, 0.49, 0.48, 0.47, 0.46, 0.45]

for k in kset:
    for part in chain:
        if len(part['cut_edges']) < k*len(initial_partition['cut_edges']):
            print(k, ' pass')
            kstar=k
            break 
    compactness_bound = constraints.UpperBound(
    lambda p: len(p['cut_edges']),
    kstar*len(initial_partition['cut_edges'])
    )
            
    chain = MarkovChain(
    proposal=proposal,
    constraints=[
        # District populations must stay within 1% of equality
        constraints.within_percent_of_ideal_population(initial_partition, 0.01),
        compactness_bound
    ],
    accept=accept.always_accept,
    initial_state=part,
    total_steps=100
    )

compactness_bound = constraints.UpperBound(
    lambda p: len(p['cut_edges']),
    kstar*len(initial_partition['cut_edges'])
)
print(kstar)

0.9  pass
0.8  pass
0.7  pass
0.6  pass
0.58  pass
0.56  pass
0.54  pass
0.52  pass
0.51  pass
0.5  pass
0.5


In [15]:
# Configure the MarkovChain by using kstar facto


chain = MarkovChain(
    proposal=proposal,
    constraints=[
        # District populations must stay within 1% of equality
        constraints.within_percent_of_ideal_population(initial_partition, 0.01),
        compactness_bound
    ],
    accept=accept.always_accept,
    initial_state=part,
    total_steps=5
)

In [16]:

import time
start_time = time.time()
reg = []
i=0
myfile = open('ILMapsdel.txt', 'w')
for part in chain:
  if random.uniform(0,1)<0.1:
    i=i+1
    onemap=list(part.assignment.values())
    myfile.write("%s\n" % onemap)
  if i==50000:
    break
myfile.close()
end_time = time.time()
print("Total_time = ", end_time-start_time, " seconds")

Total_time =  0.49031519889831543  seconds


In [ ]:
myfile.close()
fin = open("ILMapsdel.txt", "rt")
data = fin.read()
data = data.replace('[', '')
data = data.replace(']', '')
fin.close()
fin = open("ILMapsdel.txt", "wt")
fin.write(data)
fin.close()